<a href="https://colab.research.google.com/github/CSSamarasinghe/SE4050_Assignment/blob/IT21222740/%F0%9F%9B%92_5_class_Classification_Amazon_Reviews_48_%5BGRU%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **5 classes fine-grained Text Classification Amazon Reviews  GRU**


1. Loading Dataset (Amazon Reviews)
2. Download and Imports
3. Splitting it into training and testing sets.
4. Preprocessing the data (tokenizing and padding).
5. Building and training the model.
6. DTesting the model on the test dataset.


In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'amazon-reviews-for-sentianalysis-finegrained-csv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2078107%2F3499094%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240930%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240930T150806Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0384286882e57ed5979a453d3b36e2d5ac7c5683c21364a514272ce697b7a7430176301d285f6982230aa12ded3cf24f63136365e5939a1b4cf6a627f90b21b558772ac2b852ceeda8c9115147d5d45101662dbded2dc256b64c8607b23b8a2e138946585e223ab27c2e6ced66542d64b390b7a29fb5f28667574314b9b5d6a1e728981f43b957023ed099b2adfc9609101a051eb013f254b576ffd5c986f20ab81845f69f45dcde2f9c5e05f1e3fc15e3aed90b0112a274ad37e14053eb8e6d8635f649c8362c1478bc8b587d074f6474c488f679d16695f5ed6a20ddea00f6dc13ab4b1746b75013f78e36b46727d42ca74c2ba6b6d35bd214cd27c5751ebb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 654512809 bytes downloaded
Downloaded and uncompressed: amazon-reviews-for-sentianalysis-finegrained-csv
Data source import complete.


# 1. Download and Imports

In [10]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import GRU, Dense, Embedding, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [11]:
# Step 1: Load and Preprocess the Data
# Load the dataset
data = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/train.csv')


In [22]:
# Reduce the dataset to a manageable size (e.g., 25,000 samples)
data_sample = data.sample(n=100000, random_state=42)

In [23]:
# Step 2: Extract Features and Labels
X = data_sample['review_text'].values  # Features (text data)
y = data_sample['class_index'].values - 1  # Labels, adjusted to start from 0 (for class indexing)


In [24]:
# Step 3: Split the Dataset into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
# Step 4: Preprocess the Text Data (Tokenization and Padding)
# Tokenize the text (same for both train and test)
tokenizer = Tokenizer(num_words=10000)  # Limit to top 10,000 words
tokenizer.fit_on_texts(X_train)  # Fit tokenizer only on training data


In [26]:
# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


In [27]:
# Pad sequences to a uniform length (e.g., 200)
X_train_padded = pad_sequences(X_train_sequences, maxlen=200)
X_test_padded = pad_sequences(X_test_sequences, maxlen=200)


In [28]:
# Step 5: Build the GRU Model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=200))  # Embedding layer
model.add(GRU(64, return_sequences=True))  # GRU layer
model.add(GlobalMaxPool1D())  # Pooling layer
model.add(Dense(5, activation='softmax'))  # Output layer with 5 classes (1-5 rating)

In [29]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [30]:
# Step 6: Train the Model
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_data=(X_test_padded, y_test))


Epoch 1/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 355s 282ms/step - accuracy: 0.3868 - loss: 1.3805 - val_accuracy: 0.5002 - val_loss: 1.1511
Epoch 2/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 399s 296ms/step - accuracy: 0.5416 - loss: 1.0655 - val_accuracy: 0.5119 - val_loss: 1.1213
Epoch 3/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 388s 300ms/step - accuracy: 0.6008 - loss: 0.9554 - val_accuracy: 0.5096 - val_loss: 1.1329
Epoch 4/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 371s 297ms/step - accuracy: 0.6532 - loss: 0.8531 - val_accuracy: 0.5051 - val_loss: 1.1828
Epoch 5/5
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 373s 299ms/step - accuracy: 0.7080 - loss: 0.7480 - val_accuracy: 0.4882 - val_loss: 1.2516


In [31]:
# Step 7: Evaluate the Model on the Test Set
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

625/625 ━━━━━━━━━━━━━━━━━━━━ 23s 37ms/step - accuracy: 0.4839 - loss: 1.2555
Test Accuracy: 0.4882
